# Extract and analyze a movie

Industries such as Media & Entertainment, Advertising and Sports manage vast inventories of professionally produced videos, including TV shows, movies, news, sports events, documentaries, and more. To effectively extract insights from this type of video content, users require information such as video summaries, chapter-level analysis, IAB classifications for ad targeting, and speaker identification.

> [IAB categories](https://smartclip.tv/adtech-glossary/iab-categories/) are standard classifications for web content that are developed by the Interactive Advertising Bureau (IAB). These categories are used to sort advertisers into industries and segments.

In this lab, we will use BDA Video to extract and analyze a sample open-source movie [Meridian](https://en.wikipedia.org/wiki/Meridian_(film)), walking through the process and exploring the generated outputs.
![video moderation](../static/bda-video-chapter.png)

## Prerequisits

In [ ]:
%pip install "boto3>=1.37.4" "moviepy==2.1.2" --upgrade -qq

In [ ]:
import boto3
import json
import uuid
import utils
import sagemaker

bda_client = boto3.client('bedrock-data-automation')
bda_runtime_client = boto3.client('bedrock-data-automation-runtime')
s3_client = boto3.client('s3')

For a self-hosted workshop, we recommend creating a new S3 bucket in the same region where you plan to run the workshop. You can name it `bda-workshop-YOUR_ACCOUNT_ID-YOUR_REGION`.

In [ ]:
data_bucket = sagemaker.Session().default_bucket() # Enter your bucket name here
data_prefix = "bda-workshop/image"
output_prefix = "bda-workshop/image/ouput"

print('Workshop S3 bucket:', data_bucket)

In [ ]:
# Get current AWS account Id and region
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']
region = boto3.Session().region_name

print(f'Current AWS account Id: {account_id}, region name: {region}')

## Create a BDA project with a standard output configuration for videos
To start a BDA job, you need a BDA project, which organizes both standard and custom output configurations. This project is reusable, allowing you to apply the same configuration to process multiple videos that share the same settings.

In the code snippet below, we create a BDA project with standard output configurations for video modality. These configurations can be tailored to extract only the specific information you need. In this lab, we will enable the below video outputs:
- Full video summary
- Chapter summaries
- IAB categories on the chapter level
- Full audio transcript
- Text in video with bounding-boxes
- Brand & logo in video with bounding-boxes

For a complete API reference for creating a BDA project, refer to this [document](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-data-automation/client/create_data_automation_project.html).

In [ ]:
response = bda_client.create_data_automation_project(
    projectName=f'bda-workshop-video-project-{str(uuid.uuid4())[0:4]}',
    projectDescription='BDA workshop video sample project',
    projectStage='DEVELOPMENT',
    standardOutputConfiguration={
        'video': {
            'extraction': {
                'category': {
                    'state': 'ENABLED',
                    'types': ['TEXT_DETECTION','TRANSCRIPT','LOGOS'],
                },
                'boundingBox': {
                    'state': 'ENABLED',
                }
            },
            'generativeField': {
                'state': 'ENABLED',
                'types': ['VIDEO_SUMMARY','CHAPTER_SUMMARY','IAB'],
            }
        }
    }
)

The create_data_automation_project API will return the project ARN, which we will use it to invoke the video analysis task.

In [ ]:
video_project_arn = response.get("projectArn")
print("BDA video project ARN:", video_project_arn)

## Start an asynchronous BDA task to extract and analyze a movie
In this section, we will use a open-source movie Meridian, and extract and analyze it using BDA, applying the configuration defined in the BDA project. We will then review the output to gain a deeper understanding of how BDA performs video extraction and analysis.

### Prepare the sample video

In [ ]:
# Download sample video
sample_video_movie = 'NetflixMeridian.mp4'
source_url = f'https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/335119c4-e170-43ad-b55c-76fa6bc33719/NetflixMeridian.mp4'

!curl {source_url} --output {sample_video_movie}

Let's display the video. [Meridian](https://en.wikipedia.org/wiki/Meridian_(film)) is a test movie from Netflix, we use it to showcase how BDA works with video extraction. As you can see, it is a classic-style movie composed of multiple chapters.

In [ ]:
from IPython.display import Video
Video(sample_video_movie, width=800)

To analyze the video using BDA, we need to upload it to an S3 bucket that BDA can access. 

In [ ]:
s3_key = f'{data_prefix}/{sample_video_movie.split("/")[-1]}'
s3_client.upload_file(sample_video_movie, data_bucket, s3_key)

### Start BDA task
We will now invoke the BDA API to process the uploaded video. You need to provide the BDA project ARN that we created at the beginning of the lab and specify an S3 location where BDA will store the output results.

For a complete API reference for invoke a BDA async task, refer to this [document](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-data-automation-runtime/client/invoke_data_automation_async.html).

In [ ]:
response = bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={
        's3Uri': f's3://{data_bucket}/{s3_key}'
    },
    outputConfiguration={
        's3Uri': f's3://{data_bucket}/{output_prefix}'
    },
    dataAutomationConfiguration={
        'dataAutomationProjectArn': video_project_arn,
        'stage': 'DEVELOPMENT'
    },
    notificationConfiguration={
        'eventBridgeConfiguration': {
            'eventBridgeEnabled': False
        }
    },
    dataAutomationProfileArn=f'arn:aws:bedrock:{region}:{account_id}:data-automation-profile/us.data-automation-v1'
)

The `invoke_data_automation_async` API is asynchronous. It returns an invocation task identifier, `invocationArn`. We can then use another API `get_data_automation_status` to monitor the task's status until it completes.

> In production workloads, an event-driven pattern is recommended. Allow BDA to trigger the next step once the task is complete. This can be achieved by configuring the notificationConfiguration in the invoke task, which will send a notification to a subscribed AWS service, such as a Lambda function. Alternatively, you can set up an S3 trigger on the bucket where BDA will drop the results.

In [ ]:
invocation_arn = response.get("invocationArn")
print("BDA task started:", invocation_arn)

In this lab, we will use the loop below to monitor the task by calling the `get_data_automation_status` API every 5 seconds until the task is complete.

This video will take ~5-10 minutes.

In [ ]:
import time
from IPython.display import clear_output
from datetime import datetime

status, status_response = None, None
while status not in ["Success","ServiceError","ClientError"]:
    status_response = bda_runtime_client.get_data_automation_status(
        invocationArn=invocation_arn
    )
    status = status_response.get("status")
    clear_output(wait=True)
    print(f"{datetime.now().strftime('%H:%M:%S')} : BDA video task: {status}")
    time.sleep(5)

output_config = status_response.get("outputConfiguration",{}).get("s3Uri")
print("Ouput configureation file:", output_config)

## Access the BDA analysis result
The `get_data_automation_status` API returns an S3 URI containing the result configuration, which provides the S3 location where BDA outputs the extraction results. We will then parse this file to retrieve the result path.

In [ ]:
config_data = utils.read_json_on_s3(output_config,s3_client)
print(json.dumps(config_data, indent=4))

As shown above, the BDA output configuration file contains metadata about the BDA result, including the job ID, status, modality, and the S3 location of the actual result JSON. We will now download this result file to verify the output.

In [ ]:
from IPython.display import JSON

result_uri = config_data["output_metadata"][0]["segment_metadata"][0]["standard_output_path"]
result_data = utils.read_json_on_s3(result_uri,s3_client)

JSON(result_data)

## Review the result
The BDA video analysis result contains a detailed breakdown of information, organized by video and chapter level.
> A video chapter is a sequence of shots that form a coherent unit of action or narrative within the video. This feature breaks down the video into meaningful segments based on visual and audible cues, provides timestamps for those segments, and summarizes each. 

### Full video summary

Let's take a look at the video level summary - it distills the key themes, events, and information presented throughout the video into a concise summary. 

In [ ]:
print(result_data["video"]["summary"])

### Full video transcription
At the video level, we also receive the full transcript based on the video's audio, with speakers identified.

In [ ]:
print(result_data["video"]["transcript"]["representation"]["text"])

### Chapter time ranges, summaries, and IAB categories 
BDA also generates a chapter-level summary, as specified in the project configuration. Additionally, we get more metadata, including the start and end times of each chapter, as well as the [IAB](https://en.wikipedia.org/wiki/Interactive_Advertising_Bureau) categories classified based on the chapter content.

In [ ]:
for chapter in result_data["chapters"]:
    iabs = []
    if chapter.get("iab_categories"):
        for iab in chapter["iab_categories"]:
            iabs.append(iab["category"])
        
    print(f'[{chapter["start_timecode_smpte"]} - {chapter["end_timecode_smpte"]}] {", ".join(iabs)}')
    print(chapter["summary"])
    print()

### Audio transcript segements
Granular transcripts are also available at the chatper level. Under each chatper, you can find a list named `audio_segments` with associated timestamps. This can support additional downstream analysis that requires detailed transcript information.

In [ ]:
for chapter in result_data["chapters"]:
    for trans in chapter["audio_segments"]:
        print(f'[{trans["start_timestamp_millis"]/1000} - {trans["end_timestamp_millis"]/1000}] {trans["text"]}')

### Frame level text extraction with bounding-boxes and confidence scores
Text extraction, along with bounding boxes and confidence scores, is available at the frame level. In the output JSON structure, frames are organized under each chatper with captured timestamp. If text is detected at a given frame, you can find text_words and text_lines included at the frame level.

Let's plot the frames for a given chapter with detected text, including their bounding boxes.

In [ ]:
# plot all frames with boundingbox in the given chapter
chapter_index = 1
utils.plot_text(sample_video_movie, result_data, chapter_index)

### Frame level logo detection with bounding-boxes and confidence scores
Logo detection is available at the frame level, in a format similar to the text detection output. The metadata is provided at the timestamp level, including bounding boxes and confidence scores.

Let's plot the frames for a given chapter with detected logos, including their bounding boxes.

In [ ]:
# plot all frames with boundingbox in the given chapter
chapter_index = 14
utils.plot_logo(sample_video_movie, result_data, chapter_index)

### Shots in video
The BDA video returns a list of shots, each with a start and end time.
> A shot is a series of interrelated consecutive pictures taken contiguously by a single camera and representing a continuous action in time and space. 

In the following cells, we will display screenshots of all the shots, based on the start timestamps from the BDA output.

In [ ]:
# Generate shot images
images = utils.generate_shot_images(sample_video_movie, result_data, image_width=120)

# Ploat the shot images
utils.plot_shots(images)

## Summary
BDA video returns highly detailed metadata managed by the configuration. In this lab, we have enabled standard outputs required for media video analysis, using a movie as an example. You can explore the output JSON to discover more details. This lab does not cover moderation detection and analysis; for that, you can refer to the next lab, which uses a social media-style video as an example to better understand the moderation analysis offered by BDA video.

## Clean up

Delete the BDA project, blueprint, image, and result from S3.

In [ ]:
# delete BDA project
response = bda_client.delete_data_automation_project(
    projectArn=video_project_arn
)
response

In [ ]:
# delete uploaded image from S3
s3_client.delete_object(Bucket=data_bucket, Key=s3_key)

In [ ]:
# delete results from S3
utils.delete_s3_folder(data_bucket, output_config.replace("job_metadata.json","") ,s3_client)